In [ ]:
# This model takes input images of shape (224, 224, 3), and the input data should range [0, 255]. 
# Normalization is included as part of the model.
print("-------Instantiating EfficientNetB0 for testing----------")
eff_model = EfficientNetB0(weights='imagenet');

print("-------Predicting sample image----------")
img_path = 'C:\\Users\\User\\Documents\\UKM\\Project\\Models\\sampleImg\\elephant.jpg'
#EfficientNet expects image input to be of size 224x224
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

preds = eff_model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [1]:
from tensorflow.keras.applications import EfficientNetB0
INPUT_SHAPE = (224,224,3)
effB0_model = EfficientNetB0(weights='imagenet', 
                include_top=False, input_shape=INPUT_SHAPE)
effB0_model.trainable = False
effB0_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0

In [2]:
from tensorflow.keras.applications import EfficientNetB0
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import Input, Sequential, regularizers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from operator import add
import numpy as np

INPUT_SHAPE = (224,224,3)
effB0_model = EfficientNetB0(weights='imagenet', 
                include_top=False, input_shape=INPUT_SHAPE)
effB0_model.trainable = False
#effB0_model.summary()
truncEffB0Model = Model(inputs=effB0_model.input ,outputs=effB0_model.get_layer('block2b_add').output, name='efficientnetb0')

# ADD NEW TRAINABLE LAYERS ON TOP TO BUILD THE FINAL MODEL

inputs = Input(shape=(224,224,3))
x = truncEffB0Model(inputs, training=False) #run in inference mode
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
outputs = Dense(3,activation='softmax', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x) #3 output classes
model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 56, 56, 24)       20313     
                                                                 
 global_average_pooling2d (G  (None, 24)               0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 24)                0         
                                                                 
 dense (Dense)               (None, 3)                 75        
                                                                 
Total params: 20,388
Trainable params: 75
Non-trainable params: 20,313
________________________________________________________

In [5]:
from train import TrainProcedure
from losocv import LosoCv

fld = os.getcwd()
datasetFile = fld + '\dataset-csv\combinedDataset3class.csv'
weights = fld + '/trainingWeights'
trainProc = TrainProcedure()
loso = LosoCv(datasetFile)

subjects=['samm-020']

for subject in subjects:
    testSet = loso.getTestDataSet(subject)
    testSet = testSet.batch(len(testSet))
    
    print("Evaluating " + subject)
    classes = [0,1,2]
    res = model.predict(testSet)
    y_pred = [classes[np.argmax(p)] for p in res]
    #print(y_pred)
    y_actual = np.concatenate([y for x, y in testSet], axis=0)
    #print(y_actual)
    cm = confusion_matrix(y_actual,y_pred,labels=classes)
    print(cm)
    cpc = trainProc.getCountsPerClass(cm)
    print(cpc)
    resultCounts = list( map(add, resultCounts, cpc) )

Evaluating samm-020
[[0 0 2]
 [0 0 2]
 [0 0 0]]
[0, 2, 0, 0, 2, 0, 0, 0, 4]


NameError: name 'resultCounts' is not defined

In [2]:
from train import TrainProcedure
from losocv import LosoCv
from tensorflow.keras.applications import EfficientNetB0
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import Input, Sequential, regularizers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from operator import add
import numpy as np

fld = os.getcwd()
datasetFile = fld + '\dataset-csv\combinedDataset3class.csv'
weights = fld + '/trainingWeights'
trainProc = TrainProcedure()
loso = LosoCv(datasetFile)

p = './trainingWeights/3class-trunc2.h5'
model = load_model(p)
model.load_weights('./trainingWeights/smic-s19-head-trunc2epoch1-0901202221.h5')

testSet = loso.getTestDataSet('smic-s19')
testSet = testSet.batch(len(testSet))
classes = [0,1,2]
res = model.predict(testSet)
y_pred = [classes[np.argmax(p)] for p in res]
print(y_pred)
y_actual = np.concatenate([y for x, y in testSet], axis=0)
print(y_actual)

[0, 0]
[1. 2.]


In [17]:
import csv
import os
import shutil

with open('./dataset-csv/combinedDataset3class.csv') as f:
    reader = csv.reader(f, delimiter=',')
    data = list(reader) 

samples = [x[0].rsplit("/", maxsplit=1)[1] for x in data]

ori = r'C:\Users\User\Documents\UKM\Project\Dataset\Consolidated Dataset\pre-proc3class'
dest = r'C:\Users\User\Documents\UKM\Project\Dataset\Consolidated Dataset\pre-proc3classNonMegc\\'
for f in  os.listdir(ori):
    if f not in samples:
        shutil.move(ori + '\\' + f, dest + f)


In [10]:
p = r"C:\Users\User\Documents\UKM\Project\Dataset\Consolidated Dataset\pre-proc3class\casme2-sub01_EP19_05f-disgust.jpg"
p.rsplit("\\", maxsplit=1)[1]

'casme2-sub01_EP19_05f-disgust.jpg'